In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\new smote\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the CNN model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
cnn_model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten()
])
cnn_output = cnn_model(cnn_input)
cnn_output_reshaped = Reshape((1, cnn_output.shape[1]))(cnn_output)  # Reshape for LSTM

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, cnn_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(cnn_output_reshaped)

# Combine the CNN and LSTM models
combined_model = Model(inputs=cnn_input, outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=312,
    epochs=30)
# Save the trained model
combined_model.save('lung_detection_cnnlstmnotpre30.h5')


Found 16050 images belonging to 6 classes.
Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 439s 1s/step - accuracy: 0.4950 - loss: 1.2604
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 235s 751ms/step - accuracy: 0.6851 - loss: 0.8355
Epoch 3/30


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 381s 1s/step - accuracy: 0.7304 - loss: 0.7379
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 233s 746ms/step - accuracy: 0.7671 - loss: 0.6315
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 284s 892ms/step - accuracy: 0.7944 - loss: 0.5506
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 135s 432ms/step - accuracy: 0.8001 - loss: 0.5381
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 204s 649ms/step - accuracy: 0.8314 - loss: 0.4670
Epoch 8/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 215s 689ms/step - accuracy: 0.8377 - loss: 0.4470
Epoch 9/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 393s 1s/step - accuracy: 0.8447 - loss: 0.4183
Epoch 10/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 233s 744ms/step - accuracy: 0.8665 - loss: 0.3761
Epoch 11/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 393s 1s/step - accuracy: 0.8709 - loss: 0.3699
Epoch 12/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 175s 558ms/step - accuracy: 0.8706 - loss: 0.3600
Epoch 13/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 391s 1s/step - accuracy: 0.8826 - loss: 0.3322
Epoch 14/30
312/312 ━━━━━━━━━

ValueError: Unable to synchronously create dataset (name already exists)

In [5]:
# Set the path to your test dataset
test_data_dir = r"D:\new smote\test-20230326T155708Z-001\test"

# Create data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for integer labels
    shuffle=False)  # Disable shuffling to maintain the order of images

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(test_generator)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Optionally, make predictions on the test data
predictions = combined_model.predict(test_generator)


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 16s 103ms/step - accuracy: 0.8677 - loss: 0.4291
Test loss: 0.4357033371925354
Test accuracy: 0.8620437979698181
129/129 ━━━━━━━━━━━━━━━━━━━━ 15s 109ms/step


In [6]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test generator
true_labels = test_generator.classes

# Compute precision, recall, and the confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.88      0.90      0.89       685
       COVID       0.88      0.75      0.81       685
      NORMAL       0.83      0.89      0.86       685
   PNEUMONIA       0.96      0.93      0.95       685
PNEUMOTHORAX       0.71      0.78      0.74       685
TUBERCULOSIS       0.94      0.93      0.93       685

    accuracy                           0.86      4110
   macro avg       0.87      0.86      0.86      4110
weighted avg       0.87      0.86      0.86      4110

Confusion Matrix:
[[616  17   2   2  48   0]
 [ 38 512  23   3  85  24]
 [ 12   4 610  18  41   0]
 [  1   4  28 639  12   1]
 [ 28  41  67   1 532  16]
 [  9   6   8   0  28 634]]


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\new smote\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the CNN model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
cnn_model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten()
])
cnn_output = cnn_model(cnn_input)
cnn_output_reshaped = Reshape((1, cnn_output.shape[1]))(cnn_output)  # Reshape for LSTM

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, cnn_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(cnn_output_reshaped)

# Combine the CNN and LSTM models
combined_model = Model(inputs=cnn_input, outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=312,
    epochs=20)
# Save the trained model
combined_model.save('lung_detection_cnnlstmnotpre20.h5')


Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


312/312 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.4561 - loss: 1.3509
Epoch 2/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 245s 784ms/step - accuracy: 0.6846 - loss: 0.8318
Epoch 3/20


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 604s 2s/step - accuracy: 0.7228 - loss: 0.7481
Epoch 4/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 308s 984ms/step - accuracy: 0.7714 - loss: 0.6255
Epoch 5/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 499s 2s/step - accuracy: 0.7957 - loss: 0.5572
Epoch 6/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 280s 897ms/step - accuracy: 0.8120 - loss: 0.5012
Epoch 7/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 469s 1s/step - accuracy: 0.8227 - loss: 0.4754
Epoch 8/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 263s 842ms/step - accuracy: 0.8393 - loss: 0.4483
Epoch 9/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - accuracy: 0.8469 - loss: 0.4220
Epoch 10/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 381s 1s/step - accuracy: 0.8550 - loss: 0.3988
Epoch 11/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - accuracy: 0.8614 - loss: 0.3866
Epoch 12/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 361s 1s/step - accuracy: 0.8766 - loss: 0.3489
Epoch 13/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 618s 2s/step - accuracy: 0.8795 - loss: 0.3276
Epoch 14/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 

ValueError: Unable to synchronously create dataset (name already exists)

In [2]:
# Set the path to your test dataset
test_data_dir = r"D:\new smote\test-20230326T155708Z-001\test"

# Create data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for integer labels
    shuffle=False)  # Disable shuffling to maintain the order of images

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(test_generator)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Optionally, make predictions on the test data
predictions = combined_model.predict(test_generator)


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 60s 449ms/step - accuracy: 0.8536 - loss: 0.4144
Test loss: 0.41465848684310913
Test accuracy: 0.8520681262016296
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 159ms/step


In [3]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test generator
true_labels = test_generator.classes

# Compute precision, recall, and the confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.91      0.87      0.89       685
       COVID       0.86      0.75      0.80       685
      NORMAL       0.80      0.88      0.84       685
   PNEUMONIA       0.97      0.88      0.92       685
PNEUMOTHORAX       0.69      0.76      0.72       685
TUBERCULOSIS       0.91      0.96      0.94       685

    accuracy                           0.85      4110
   macro avg       0.86      0.85      0.85      4110
weighted avg       0.86      0.85      0.85      4110

Confusion Matrix:
[[596  23   4   1  57   4]
 [ 24 517  19   4  91  30]
 [  4   4 605  10  55   7]
 [  2   4  57 604  12   6]
 [ 24  51  69   3 523  15]
 [  2   5   0   1  20 657]]


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\NEW_SMOTE\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the CNN model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
cnn_model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten()
])
cnn_output = cnn_model(cnn_input)
cnn_output_reshaped = Reshape((1, cnn_output.shape[1]))(cnn_output)  # Reshape for LSTM

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, cnn_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(cnn_output_reshaped)

# Combine the CNN and LSTM models
combined_model = Model(inputs=cnn_input, outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=312,
    epochs=10)



Found 16050 images belonging to 6 classes.
Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 823s 3s/step - accuracy: 0.4865 - loss: 1.2676
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 320s 1s/step - accuracy: 0.6977 - loss: 0.8013
Epoch 3/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 491s 2s/step - accuracy: 0.7355 - loss: 0.7084
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 252s 806ms/step - accuracy: 0.7805 - loss: 0.5968
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 414s 1s/step - accuracy: 0.7956 - loss: 0.5593
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 228s 729ms/step - accuracy: 0.8396 - loss: 0.4588
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 391s 1s/step - accuracy: 0.8257 - loss: 0.4719
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 234s 748ms/step - accuracy: 0.8386 - loss: 0.4444
Epoch 9/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 393s 1s/step - accuracy: 0.8425 - loss: 0.4230
Epoch 10/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 238s 760ms/step - accuracy: 0.8641 - loss: 0.3673


In [6]:
# Set the path to your test dataset
test_data_dir = r"D:\NEW_SMOTE\test-20230326T155708Z-001\test"

# Create data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for integer labels
    shuffle=False)  # Disable shuffling to maintain the order of images

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(test_generator)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Optionally, make predictions on the test data
predictions = combined_model.predict(test_generator)


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 33s 258ms/step - accuracy: 0.8188 - loss: 0.4982
Test loss: 0.5001562237739563
Test accuracy: 0.8158150911331177
129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 107ms/step


In [7]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test generator
true_labels = test_generator.classes

# Compute precision, recall, and the confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.89      0.80      0.84       685
       COVID       0.70      0.85      0.77       685
      NORMAL       0.91      0.68      0.78       685
   PNEUMONIA       0.95      0.89      0.92       685
PNEUMOTHORAX       0.62      0.78      0.69       685
TUBERCULOSIS       0.96      0.90      0.93       685

    accuracy                           0.82      4110
   macro avg       0.84      0.82      0.82      4110
weighted avg       0.84      0.82      0.82      4110

Confusion Matrix:
[[545  74   0   1  62   3]
 [ 21 584   5   0  58  17]
 [ 11  24 466  34 148   2]
 [  8  24  18 609  25   1]
 [ 25  98  23   0 535   4]
 [  3  26   1   0  41 614]]
